# Importar librerías

In [4]:
"""
Generador de BBDD y datos sintéticos para repostajes de autobuses y camiones
Output: CSVs (fuel_prices_monthly.csv, electricity_prices_monthly.csv, vehicle_catalog.csv, refuel_events.csv)
Requiere: pandas, numpy
"""

import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
import random

RND = np.random.RandomState(42)

OUT_DIR = Path("output_data")
OUT_DIR.mkdir(exist_ok=True)

# Agrupar series diarias a mensuales

In [5]:
def daily_to_monthly(df, date_col, value_cols, agg_funcs=None):
    """
    df: dataframe con columna fecha (datetime)
    value_cols: list of columns with numeric values to aggregate
    agg_funcs: dict col->agg function, default min,mean,max
    """
    df = df.copy()
    df['year'] = df[date_col].dt.year
    df['month'] = df[date_col].dt.month
    if agg_funcs is None:
        agg_funcs = {c: ['min','mean','max'] for c in value_cols}
    monthly = df.groupby(['year','month']).agg(agg_funcs)
    # flatten columns
    monthly.columns = ['_'.join([c, f]) for c, f in monthly.columns]
    monthly = monthly.reset_index()
    return monthly

# Lectura csv

In [6]:
def load_cnmc_daily_prices(csv_path):
    """
    Espera CSV con columnas: date (YYYY-MM-DD), province, product (e.g. 'Gasóleo A'), price_eur_per_l
    (Adapta según el CSV oficial que descargues de data.cnmc.es)
    """
    df = pd.read_csv(csv_path, parse_dates=['date'])
    # Normalizar nombres de columnas si hace falta
    return df

def build_monthly_fuel_prices_from_daily(df_daily):
    # df_daily should have: date, product, price_eur_per_l, province (optional)
    monthly = df_daily.rename(columns={'date':'fecha','price_eur_per_l':'price'}).copy()
    monthly['fecha'] = pd.to_datetime(monthly['fecha'])
    out = monthly.groupby([monthly['fecha'].dt.year, monthly['fecha'].dt.month, 'product'])['price'].agg(['min','mean','max']).reset_index()
    out.columns = ['year','month','product','min_eur_per_l','mean_eur_per_l','max_eur_per_l']
    return out

# Generador vehículos

In [7]:
def generate_vehicle_catalog(n_trucks=200, n_buses=100, frac_electric_buses=0.15):
    rows = []
    vid = 1
    # Trucks (diesel / LNG mixture)
    for i in range(n_trucks):
        tank = int(RND.choice([300,400,500,600,800,1000], p=[0.05,0.1,0.25,0.25,0.2,0.15]))
        cons = float(RND.normal(loc=30, scale=5))  # L/100km
        rows.append({
            'vehicle_id': f'T{vid:05d}',
            'vehicle_type':'truck',
            'fuel_type':'diesel',
            'tank_capacity_l': tank,
            'battery_capacity_kwh': np.nan,
            'consumption_l_per_100km': round(max(12, cons),2),
            'consumption_kwh_per_km': np.nan
        })
        vid += 1
    # Buses: mix diesel / electric
    n_elect = int(n_buses * frac_electric_buses)
    for i in range(n_buses):
        if i < n_elect:
            # electric bus
            batt = int(RND.choice([200,250,300,350,400], p=[0.1,0.25,0.3,0.2,0.15]))
            cons_kwh_per_km = float(RND.normal(loc=1.5, scale=0.2))
            rows.append({
                'vehicle_id': f'B{vid:05d}',
                'vehicle_type':'city_bus',
                'fuel_type':'electric',
                'tank_capacity_l': np.nan,
                'battery_capacity_kwh': batt,
                'consumption_l_per_100km': np.nan,
                'consumption_kwh_per_km': round(max(0.8, cons_kwh_per_km),3)
            })
        else:
            # diesel bus
            tank = int(RND.choice([200,250,300,350,400,500], p=[0.05,0.2,0.25,0.2,0.15,0.15]))
            cons = float(RND.normal(loc=40, scale=8))  # L/100km
            rows.append({
                'vehicle_id': f'B{vid:05d}',
                'vehicle_type':'city_bus',
                'fuel_type':'diesel',
                'tank_capacity_l': tank,
                'battery_capacity_kwh': np.nan,
                'consumption_l_per_100km': round(max(15, cons),2),
                'consumption_kwh_per_km': np.nan
            })
        vid += 1
    df = pd.DataFrame(rows)
    df.to_csv(OUT_DIR/"vehicle_catalog.csv", index=False)
    return df

# Generador sintético evento repostaje

In [ ]:
def simulate_refuels(vehicle_catalog, start_date='2018-01-01', end_date=None, avg_refuels_per_month=4):
    if end_date is None:
        end_date = datetime.now().date()
    start = pd.to_datetime(start_date).date()
    end = pd.to_datetime(end_date).date()
    days = (end - start).days
    events = []
    odo_base = 100000 # odometro inicial aproximado
    for _, v in vehicle_catalog.iterrows():
        # decide number of refuels for this vehicle in period
        months = max(1, days/30.0)
        n_events = int(max(1, RND.poisson(lam=avg_refuels_per_month*months/12.0))) if v.vehicle_type=='truck' else int(max(1, RND.poisson(lam=avg_refuels_per_month*months/12.0)))
        last_odo = odo_base + RND.randint(0,200000)
        for e in range(n_events):
            # random date in interval (uniform)
            dt = start + timedelta(days=int(RND.randint(0,days)))
            if v.fuel_type == 'diesel':
                # liters: if 'full' strategy fill 60-95% of tank; else top-up (10-50%)
                fill_type = RND.choice(['full','topup'], p=[0.6,0.4])
                if fill_type=='full':
                    liters = float(v.tank_capacity_l) * RND.uniform(0.6,0.95)
                else:
                    liters = float(v.tank_capacity_l) * RND.uniform(0.1,0.5)
                km_since_last = liters / (v.consumption_l_per_100km/100.0) \
                    if not np.isnan(v.consumption_l_per_100km) else int(RND.normal(300,100))
                kwh = np.nan
            else:
                # electric
                batt = v.battery_capacity_kwh
                # charge can be partial: 30-90% of battery
                soc_gain = RND.uniform(0.3,0.9)
                kwh = batt * soc_gain
                liters = np.nan
                km_since_last = kwh / max(0.8, v.consumption_kwh_per_km)
            # price placeholders (to be joined with monthly price tables)
            price_per_l = np.nan
            price_per_kwh = np.nan
            cost = np.nan
            last_odo += max(50, int(km_since_last))
            events.append({
                'vehicle_id': v.vehicle_id,
                'datetime': pd.to_datetime(dt),
                'location_province': RND.choice(['Madrid','Barcelona','Valencia','Sevilla','A Coruña','Zaragoza']),
                'liters': round(liters,2) if not np.isnan(liters) else np.nan,
                'kwh': round(kwh,2) if not np.isnan(kwh) else np.nan,
                'price_per_l': price_per_l,
                'price_per_kwh': price_per_kwh,
                'cost_eur': cost,
                'km_since_last': int(km_since_last),
                'odo_km': last_odo
            })
    df_events = pd.DataFrame(events).sort_values('datetime').reset_index(drop=True)
    df_events.to_csv(OUT_DIR/"refuel_events_raw.csv", index=False)
    return df_events

# Join con precios mensuales reales (en caso de tenerlos)

In [9]:
def price_join_and_calc_cost(refuels_df, fuel_price_monthly_df, elec_price_monthly_df):
    df = refuels_df.copy()
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    # join fuel prices
    # fuel_price_monthly_df must have columns year,month,product,mean_eur_per_l
    # map product by vehicle fuel_type (assumption simple)
    df = df.merge(fuel_price_monthly_df.rename(columns={'product':'product','mean_eur_per_l':'mean_eur_per_l'}),
                  how='left', left_on=['year','month'], right_on=['year','month'])
    # If kwh charges, join electricity price
    df = df.merge(elec_price_monthly_df[['year','month','mean_eur_per_kwh']].drop_duplicates(), how='left', on=['year','month'])
    # compute costs
    df['cost_eur'] = df['liters'] * df['mean_eur_per_l'].fillna(0) + df['kwh'] * df['mean_eur_per_kwh'].fillna(0)
    # fill price_per_l / price_per_kwh from month
    df['price_per_l'] = df['mean_eur_per_l']
    df['price_per_kwh'] = df['mean_eur_per_kwh']
    df.to_csv(OUT_DIR/"refuel_events_with_prices.csv", index=False)
    return df

# Ejemplo de uso

In [10]:
if __name__ == "__main__":
    vc = generate_vehicle_catalog(n_trucks=300, n_buses=120, frac_electric_buses=0.2)
    refuels = simulate_refuels(vc, start_date='2016-01-01', end_date='2025-09-20', avg_refuels_per_month=3)
    print("Vehículos:", len(vc), "Eventos simulados:", len(refuels))
    # En caso de que tengas CSVs reales:
    # fuel_daily_df = load_cnmc_daily_prices("cnmc_daily.csv")
    # fuel_monthly = build_monthly_fuel_prices_from_daily(fuel_daily_df)
    # elec_monthly = ... # cargar / construir con formato year,month,mean_eur_per_kwh
    # joined = price_join_and_calc_cost(refuels, fuel_monthly, elec_monthly)

Vehículos: 420 Eventos simulados: 12553
